<a href="https://colab.research.google.com/github/pudasainishushant/covidNepaliTweet/blob/main/covidBERT(alt).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
import torch
import pandas as pd
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df = pd.read_csv("/content/covid19_tweeter_dataset.csv")

In [ ]:
df.head()

,Unnamed: 0,Label,Datetime,Tweet,Tokanize_tweet,Unnamed: 5
0,0,-1,2021-01-10 22:06:41+00:00,अमेरिकामा कोभिड बाट एकै दिन चार हजारभन्दा बढीक...,"अमेरिकामा,कोभिड,बाट,एकै,दिन,चार,हजारभन्दा,बढीक...",NaN
1,1,-1,2021-01-10 17:49:34+00:00,कोभिड का कारण विदेशमा रहेका नेपालीहरुमा मानसिक...,"कोभिड,का,कारण,विदेशमा,रहेका,नेपालीहरुमा,मानसिक...",NaN
2,2,1,2021-01-10 16:18:34+00:00,नेपालमा क्लोभर बायोफार्मास्युटिकल्स अस्ट्रेलिय...,"नेपालमा,क्लोभर,बायोफार्मास्युटिकल्स,अस्ट्रेलिय...",NaN
3,3,0,2021-01-10 15:12:17+00:00,कोभिड को खोप पनि लगाइयो,"कोभिड,को,खोप,पनि,लगाइयो",NaN
4,4,-1,2021-01-10 15:07:12+00:00,अमेरिकामा कोभिड को नयाँ रेकर्ड एकै दिन हजारभन्...,"अमेरिकामा,कोभिड,को,नयाँ,रेकर्ड,एकै,दिन,हजारभन्...",NaN


In [ ]:
df['Tweet'].value_counts()
possible_labels = df.Tweet.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'अमेरिकामा कोभिड बाट एकै दिन चार हजारभन्दा बढीको मृत्यु': 0,
 'कोभिड का कारण विदेशमा रहेका नेपालीहरुमा मानसिक स्वास्थ्य सम्बन्धि समस्या देखिएको र आत्महत्याका घटनाहरु बढेको डा सापकोटाले बताए': 1,
 'नेपालमा क्लोभर बायोफार्मास्युटिकल्स अस्ट्रेलिया तथा भ्याक्जाइन पीटीवाई लिमिटेड अस्ट्रेलियाले उत्पादन गरेका कोभिड खोपको तेस्रो चरणका ट्रायल गर्न तयारी भइरहेको छ': 2,
 'कोभिड को खोप पनि लगाइयो': 3,
 'अमेरिकामा कोभिड को नयाँ रेकर्ड एकै दिन हजारभन्दा बढीको मृत्यु विश्व समाचार': 4,
 'गण्डकी प्रदेश सरकारले कोभिड बाट प्रभावीतहरुको लागि भनेर स्थापित व्यवसाय जीवन रक्षा कोषमा अर्ब छुट्याइएकोमा महिना लागि सक्दा पनि कार्यविधि समेत बनेको छैन यसबाट बजेट कार्यन्वयनको पक्ष कस्तो छ अनुमान लगाउन सकिन्छ': 5,
 'नेपालको संचार अमेरिकामा कोभिड को नयाँ रेकर्ड एकै दिन हजारभन्दा बढीको मृत्यु': 6,
 'रामेछापमा कोभिड सङ्क्रमितको संख्या पुग्यो': 7,
 'कोरोना भाइरस भारत माघ गतेदेखि कोभिड विरुद्ध राष्ट्रव्यापी खोप अभियान': 8,
 'स्वास्थ्य मन्त्रालयले माग्यो कोभिड को रोकथामका लागि खटिएका स्वास्थ्यकर्मी तथा कर्मचारीको विवरण': 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(df.index.values, df.Label.values, test_size=0.15, random_state=42,
stratify=df.Label.values)
df['data_type'] = ['not_set']*df.shape[0]
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'
df.groupby(['Tweet', 'Label', 'data_type']).count()

,,,Unnamed: 0,Datetime,Tokanize_tweet,Unnamed: 5
Tweet,Label,data_type,,,,
2020-11-03 14:03:28+00:00,1,val,1,1,1,1
अ ने क्रान्तिकारी ट्रेड युनियन महासंघको कार्यलयको विस्तारित बैठक सम्पन्न बैठकको निष्कर्ष क कोभिड को त्रासदी र जारी लकडाउनले विश्व मजदुर वर्ग संकटपुर्ण स्थितिमा ख रोग भन्दा भोक निर्णायक बन्दै ग मजदुरहरुलाई तत्काल विशेष राहतको घोषणाको माग घ अब आन्दोलनको तयारी,-1,train,1,1,1,0
अ लक डाउन पुर्ण पालान नगरी घर बाहिर निक्लने अनावश्यक भेलाहुने गर्नु भो भने तपाईलाई मृत्यु दण्ड को सजाय दिन कोराना कोभिड स्वयम आउने छ,-1,train,1,1,1,0
अंकहरू दोब्बर हुँदै गएका छन् स्थिति कम्तीमा अझ तीन महिना नियन्त्रणमा रहँदैन भनेर अन्तर्राष्ट्रिय अर्थशास्त्रीहरूले बताइ रहेका छन् स्वास्थ क्षेत्रका विज्ञहरूले कोभिड विरुद्धको खोप बन्न महिनासम्म लाग्ने बताइरहेका छन् स्थिति जटिल छ हामीले सोचेभन्दा जीवन र जीविका बचाउँ,1,train,1,1,1,0
अंग्रेज़ी नयॉं वर्ष ले लेराएको कोरोना भाईरस कोभिड नेपाली नयाँ वर्ष ले अन्त्य गरोस भन्दै नयॉं वर्षको मंगलमय हार्दिक शुभकामना,1,train,1,1,1,0
...,...,...,...,...,...,...
ह्वावेको एआईमा आधारित प्रविधिहरु कोभिड विरुद्ध लड्न एशिया प्यासिफिकमा उपलब्ध,1,val,1,1,1,0
ुमन्त्रिपरिषद बैठकले लकडाउनमा थप कडाइ गर्दै एक प्रदेशबाट अर्को प्रदेश एक जिल्लाबाट अर्को जिल्ला र एक स्थानीय तहबाट अर्को स्थानीय तहमा जान नदिने निर्णय गरेको हो सरकारले कैलाली कञ्चनपुर र बागलुङमा कोरोना भाइरस कोभिड को र् यापिड टेष्ट गर्ने भएको छ,1,train,1,1,1,0
ॐ कोभिड सँग लड्न सक्ने शक्ति प्राप्त हाेस चैते दशैकाे सबैमा शुभकामना,1,train,1,1,1,0


In [ ]:
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder() 
df['Label'] = lb.fit_transform(df['Label'])

In [ ]:
df

,Unnamed: 0,Label,Datetime,Tweet,Tokanize_tweet,Unnamed: 5,data_type
0,0,0,2021-01-10 22:06:41+00:00,अमेरिकामा कोभिड बाट एकै दिन चार हजारभन्दा बढीक...,"अमेरिकामा,कोभिड,बाट,एकै,दिन,चार,हजारभन्दा,बढीक...",NaN,train
1,1,0,2021-01-10 17:49:34+00:00,कोभिड का कारण विदेशमा रहेका नेपालीहरुमा मानसिक...,"कोभिड,का,कारण,विदेशमा,रहेका,नेपालीहरुमा,मानसिक...",NaN,val
2,2,2,2021-01-10 16:18:34+00:00,नेपालमा क्लोभर बायोफार्मास्युटिकल्स अस्ट्रेलिय...,"नेपालमा,क्लोभर,बायोफार्मास्युटिकल्स,अस्ट्रेलिय...",NaN,train
3,3,1,2021-01-10 15:12:17+00:00,कोभिड को खोप पनि लगाइयो,"कोभिड,को,खोप,पनि,लगाइयो",NaN,train
4,4,0,2021-01-10 15:07:12+00:00,अमेरिकामा कोभिड को नयाँ रेकर्ड एकै दिन हजारभन्...,"अमेरिकामा,कोभिड,को,नयाँ,रेकर्ड,एकै,दिन,हजारभन्...",NaN,train
...,...,...,...,...,...,...,...
33469,33469,1,2020-02-12 01:33:08+00:00,कोरोना भाइरसलाई विश्व स्वास्थ्य संगठनले दियो न...,"कोरोना,भाइरसलाई,विश्व,स्वास्थ्य,संगठनले,नाम,कोभिड",NaN,train
33470,33470,1,2020-02-12 00:38:07+00:00,डब्ल्युएचओले दियाेको कोरोनाको नयाँ नाम कोभिड,"डब्ल्युएचओले,दियाेको,कोरोनाको,नाम,कोभिड",NaN,train
33471,33471,1,2020-02-11 19:37:26+00:00,कोरोना भाइरस संक्रमणको औपचारिक नाम अब कोभिड,"कोरोना,भाइरस,संक्रमणको,औपचारिक,नाम,कोभिड",NaN,train
33472,33472,1,2020-02-11 16:36:40+00:00,विश्व स्वास्थ्य संगठन डब्लुएचओ ले नोबल कोरोना ...,"विश्व,स्वास्थ्य,संगठन,डब्लुएचओ,नोबल,कोरोना,भाइ...",NaN,train


In [ ]:
df.head()

,Unnamed: 0,Label,Datetime,Tweet,Tokanize_tweet,Unnamed: 5,data_type
0,0,0,2021-01-10 22:06:41+00:00,अमेरिकामा कोभिड बाट एकै दिन चार हजारभन्दा बढीक...,"अमेरिकामा,कोभिड,बाट,एकै,दिन,चार,हजारभन्दा,बढीक...",NaN,train
1,1,0,2021-01-10 17:49:34+00:00,कोभिड का कारण विदेशमा रहेका नेपालीहरुमा मानसिक...,"कोभिड,का,कारण,विदेशमा,रहेका,नेपालीहरुमा,मानसिक...",NaN,val
2,2,2,2021-01-10 16:18:34+00:00,नेपालमा क्लोभर बायोफार्मास्युटिकल्स अस्ट्रेलिय...,"नेपालमा,क्लोभर,बायोफार्मास्युटिकल्स,अस्ट्रेलिय...",NaN,train
3,3,1,2021-01-10 15:12:17+00:00,कोभिड को खोप पनि लगाइयो,"कोभिड,को,खोप,पनि,लगाइयो",NaN,train
4,4,0,2021-01-10 15:07:12+00:00,अमेरिकामा कोभिड को नयाँ रेकर्ड एकै दिन हजारभन्...,"अमेरिकामा,कोभिड,को,नयाँ,रेकर्ड,एकै,दिन,हजारभन्...",NaN,train


In [ ]:
tokenizer = BertTokenizer.from_pretrained('Shushant/nepaliBERT')

encoded_data_train = tokenizer.batch_encode_plus(
  df[df.data_type=='train'].Tweet.values,
  add_special_tokens=True,
  return_attention_mask=True,
  padding='max_length',  
  max_length=61, 
  truncation=True,
  return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
  df[df.data_type=='val'].Tweet.values,
  add_special_tokens=True,
  padding='max_length',  
  return_attention_mask=True,
  pad_to_max_length=True,
  max_length=61,
  truncation=True,
  return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].Label.values)
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].Label.values)
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Downloading:   0%|          | 0.00/529k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/589 [00:00<?, ?B/s]

In [ ]:
model = BertForSequenceClassification.from_pretrained("Shushant/nepaliBERT",
  num_labels=3,
  output_attentions=False,
  output_hidden_states=False)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at Shushant/nepaliBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Shushant/nepaliBERT and a

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32
dataloader_train = DataLoader(dataset_train,
                              sampler=RandomSampler(dataset_train),
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val,
                                   sampler=SequentialSampler(dataset_val),
                                   batch_size=batch_size)

In [ ]:

from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(
    model.parameters(),
    lr=1e-5,
    eps=1e-8)

epochs = 5

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(dataloader_train)*epochs
  )

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
from sklearn.metrics import f1_score
import numpy as np

def f1_score_func(preds, Labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = Labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random

seed_val = 57
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2],
        }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis = 0)
    true_vals = np.concatenate(true_vals, axis = 0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc = 'Epoch {:1d}'.format(epoch), leave = False, disable = False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2],
        }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/890 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.6849223212579663
Validation loss: 0.6146221250105816
F1 Score (Weighted): 0.7329956993083114


Epoch 2:   0%|          | 0/890 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.551194996169109
Validation loss: 0.6068027042279578
F1 Score (Weighted): 0.7486648706625403


...


Epoch 3
Training loss: 0.4745665830507707
Validation loss: 0.6439144245948002
F1 Score (Weighted): 0.7481675489690359


Epoch 4:   0%|          | 0/890 [00:00<?, ?it/s]


Epoch 4
Training loss: 0.41359097960122515
Validation loss: 0.6593769520141517
F1 Score (Weighted): 0.7516921393024343


Epoch 5:   0%|          | 0/890 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.3692217136283269
Validation loss: 0.6884667853451079
F1 Score (Weighted): 0.7528114579586807
